In [1]:
import pandas as pd
import numpy as np
import os
import pickle

from nltk.tokenize import sent_tokenize, word_tokenize
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/timcha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df_final = pd.read_csv('data/final_books.csv')
df_final.head()

,mark,publication_date,file_text,author,title,год попадания,автор/авторы,книга,цикл,аннотация,...,время,Ссылка на выпуск обзора.1,"год попадания, из исходной редакции, без ""округления""",Unnamed: 22,Unnamed: 23,Unnamed: 24,Служебные отметки,LC,link_domain,num_pers
0,3.6*84,-,Лернер_Марик_Совсем_не_прогрессор.txt,Лернер Марик,Совсем не прогрессор,2000,Лернер Марик,Совсем не прогрессор,NaN,Нормальный человек не стремится непременно все...,...,NaN,#Н/Д,2000.0,NaN,NaN,NaN,+++++,NaN,flibusta.is,398
1,1.5*31,-,Нилан_Ник_Белый_архив.txt,Нилан Ник,Белый архив,1990,Нилан Ник,Белый архив,NaN,"Он скатился на дно, залез в долги, ввязался в ...",...,NaN,#Н/Д,1990.0,NaN,NaN,NaN,+++++,NaN,flibusta.is,178
2,4.3*3,-,Малов_Владимир_Зачет_по_натуральной_истории.txt,Малов Владимир,Зачет по натуральной истории,1989,Малов Владимир,Зачет по натуральной истории,NaN,Урок ботаники в обычном классе обычной московс...,...,NaN,https://zen.yandex.ru/media/backinussr/novosti...,NaN,NaN,NaN,NaN,+++++,NaN,flibusta.is,93
3,1.4*7,-,Соловьев_Роман_Сквозь_время.txt,Соловьев Роман,Сквозь время,1989,Соловьев Роман,Сквозь время,NaN,"Мог ли подумать Сергей Павлович Кузнецов, когд...",...,NaN,https://zen.yandex.ru/media/backinussr/novosti...,NaN,NaN,NaN,NaN,+++++,NaN,flibusta.is,207
4,1.8*21,-,Рагорин_Алексей_Ракетчики.txt,Рагорин Алексей,Ракетчики,1988,Рагорин Алексей,Ракетчики,NaN,"Колониализм, общество разврата и потребления д...",...,NaN,#Н/Д,1988.0,NaN,NaN,NaN,+++++,NaN,flibusta.is,1612


In [4]:
folder_lemmas = 'data/lemmatized_books_fixed' 
book2lemmas = {}

for file in os.listdir(folder_lemmas): 
    with open(os.path.join(folder_lemmas, file), 'rb') as f: 
        data = pickle.load(f)
        
        book2lemmas |= data 
        
print(len(book2lemmas.keys()))

628


In [9]:
list(book2lemmas.values())[0]

[['двадцатый',
  'число',
  'июнь',
  'год',
  'вплотную',
  'подойти',
  'момент',
  'умный',
  'человек',
  'стать',
  'очевидный',
  'ничто',
  'прежде',
  'первый',
  'фактор',
  'радикальный',
  'перемена',
  'мировой',
  'политика',
  'стать',
  'русско',
  'японский',
  'война',
  'весы',
  'который',
  'склоняться',
  'отнюдь',
  'польза',
  'нападать',
  'сторона',
  'нападение',
  'найти',
  'тайный',
  'поддержка',
  'называть',
  'прогрессивный',
  'человечество',
  'якобы',
  'союзный',
  'российский',
  'империя',
  'французский',
  'республика',
  'свой',
  'порт',
  'расположить',
  'восточный',
  'цейлон',
  'ввести',
  'русский',
  'военный',
  'корабль',
  'правило',
  'час',
  'время',
  'крейсер',
  'броненосец',
  'должный',
  'покинуть',
  'порт',
  'противный',
  'случай',
  'оказаться',
  'интернированный'],
 ['казаться',
  'российский',
  'империя',
  'попасть',
  'полный',
  'изоляция',
  'активно',
  'враждебный',
  'британский',
  'империя',
  'международны

In [10]:
folder_books = 'data/books'

books = []
books_lemmas = [] 

for i, row in tqdm(df_final.iterrows()):
    file_text = row['file_text']
    
    # preparing texts for embeddings 
    with open(os.path.join(folder_books, file_text), 'r') as f:
        book = f.readlines()
        
        for paragraph in book: 
            books.append(paragraph.strip()) 
            
    # preparing texts for vectorizer 
    book = book2lemmas[file_text] 
    books_lemmas.extend(book)

629it [00:02, 263.61it/s]


In [13]:
len(books), len(books_lemmas)

(1372819, 1372819)

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = SentenceTransformer("DeepPavlov/rubert-base-cased-sentence")
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
embeddings = model.encode(books,
                          show_progress_bar=True
                         )

In [ ]:
with open('/content/drive/MyDrive/vkr/embeddings.npy', 'wb') as f:
    np.save(f, embeddings)

In [ ]:
# embeddings = np.load('/content/drive/MyDrive/vkr/embeddings.npy')

In [ ]:
embeddings.shape

(1372819, 312)

## Обучаем BERTopic

In [ ]:
import collections
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN
from bertopic import BERTopic

In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stopwords = list(stopwords.words('russian'))
stopwords.append('глава')
stopwords.append('очень')
stopwords.append('всё') 
stopwords.append('это')
stopwords.append('пока')
stopwords.append('просто')
stopwords.append('это') 
stopwords.append('наш')

In [ ]:
# Extract vocab to be used in BERTopic
vocab = collections.Counter()
tokenizer = CountVectorizer().build_tokenizer()
for paragraph in tqdm(books_lemmas):
    vocab.update(tokenizer(paragraph))

vocab = [word for word, frequency in vocab.items() if frequency >= 10]

len(vocab)

100%|██████████| 1372819/1372819 [00:35<00:00, 38228.80it/s]


210933

In [ ]:
from bertopic.representation import KeyBERTInspired 

keybert_model = KeyBERTInspired() 
representation_model = {
    'KeyBERT': keybert_model
}

In [ ]:
umap_model = UMAP(n_components=15, n_neighbors=20, random_state=42, metric="cosine", verbose=True)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, prediction_data=True, min_cluster_size=30, verbose=True)
vectorizer_model = CountVectorizer(vocabulary=vocab, stop_words=stopwords, ngram_range=(1, 3))

topic_model = BERTopic(
    embedding_model=model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    representation_model=representation_model, 
    
    top_n_words=10, 
    diversity=0.5,
    verbose=True
).fit(books_lemmas, embeddings=embeddings)

In [ ]:
similar_topics = topic_model.find_topics('merge', top_n=10) 
topic_model.merge_topics(books_lemmas, similar_topics)

topic_model.reduce_outliers(books_lemmas)

In [ ]:
topic_model.get_topic_info()